In [1]:
import numpy as np
import cv2

import PIL.Image as Image
import os

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.models import Sequential

import pathlib

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.layers import Concatenate

In [2]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SIZE = (224,224)
IMG_SHAPE = IMG_SIZE + (3,)


In [3]:
data_dir = pathlib.Path('./Image data')
images=list(data_dir.glob('*/*.tif'))

In [4]:
image_dic= {
    'normal' : list(data_dir.glob("e0_Normal/*")),
    'hole':list(data_dir.glob("e1_hole/*")),
    'stain' : list(data_dir.glob("e2_Stain/*")),
    'net' :list(data_dir.glob("e3_Net/*")),
    'color' :list(data_dir.glob("e4_colour/*")),
    'crease': list(data_dir.glob("e5_Crease/*")),
}


In [5]:
labels = {
    'normal':0,
    'hole':1,
    'stain':2,
    'net':3,
    'color':4,
    'crease':5,
}

In [6]:
X,y=[],[]

for name, images in image_dic.items():
    for image in images:
        img = cv2.imread(str(image))
        
        
        if img is None:
            print('Wrong path:', image)
        else:
            resized = cv2.resize(img,IMG_SIZE)
            X.append(resized)
            y.append(labels[name])

Wrong path: Image data/e0_Normal/test1.tif.gz
Wrong path: Image data/e0_Normal/test.tif.gz
Wrong path: Image data/e0_Normal/test2.gif.gz


In [7]:
X = np.array(X)/255
y= np.array(y)
X_train, X_test, y_train,y_test = train_test_split(X,y,random_state=0, shuffle=True,test_size=0.2)


In [8]:
X.shape

(1204, 224, 224, 3)

In [23]:
metrics=[tf.keras.metrics.CategoricalAccuracy(),
                           #tf.keras.metrics.BinaryAccuracy(),
                           tf.keras.metrics.Precision(),
                           tf.keras.metrics.Recall(),
                           tf.keras.metrics.TopKCategoricalAccuracy(k=2),
                           #tf.keras.metrics.AUC(num_thresholds=200,
                           #                    multi_label=True),
                           #tf.keras.metrics.TruePositives(),
                           #tf.keras.metrics.FalsePositives(),
                           ]

In [28]:
model1.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 56, 56, 3)]  0           []                               
                                                                                                  
 Conv1 (Conv2D)                 (None, 28, 28, 32)   864         ['input[0][0]']                  
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 28, 28, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 28, 28, 32)   0           ['bn_Conv1[0][0]']               
                                                                               

In [33]:
classes = 6
input_shape=(224,224,3)
input_img = tf.keras.Input(input_shape, name='input')
model1= tf.keras.applications.mobilenet.MobileNet(input_tensor = input_img, include_top= False)
model_mn = tf.keras.Sequential()

for layers in model1.layers[:-26]:
    model_mn.add(layers)


def by_pass(model):
    model.add(tf.keras.layers.GlobalAveragePooling2D())
    model.add(tf.keras.layers.Dense(100, activation='tanh'))
    model.add(tf.keras.layers.Dropout(0.6))
    model.add(tf.keras.layers.Dense(60,activation='tanh',name='predictions'))
    model.add(tf.keras.layers.Dense(6, activation='softmax'))
    return model
    


def ensemble(model_mn):
    com_out= model_mn.layers[-1].output
    seps  =[]
    for i in range(classes):
        res=tf.keras.layers.GlobalAveragePooling2D()(com_out)
        res = tf.keras.layers.Dense(20, activation='tanh')(res)
        res = tf.keras.layers.Dropout(0.4)(res)
        res= tf.keras.layers.Dense(10,activation='tanh',name='predictions'+str(i))(res)
        res = tf.keras.layers.Dense(1, activation='sigmoid')(res)

        seps.append(res)
    out= Concatenate()(seps)
    model = tf.keras.Model(model1.input,out)  


    
    return model

#model = by_pass(model_mn)
model = ensemble(model_mn)



#out = tf.keras.layers.Activation(activations.relu)(out)



for layers in model.layers[:53]:
    layers.trainable= False

model.compile(optimizer = Adam(learning_rate=0.001), loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics=metrics,)

TypeError: Tensor.__init__() missing 1 required positional argument: 'value_index'

In [25]:
history = model.fit(X_train,to_categorical(y_train),epochs=80, validation_data=(X_test,to_categorical(y_test)))

Epoch 1/80


2023-02-26 20:44:27.866535: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


31/31 [==============================] - ETA: 0s - loss: 1.7747 - categorical_accuracy: 0.2170 - precision_1: 0.1878 - recall_1: 0.3853 - top_k_categorical_accuracy: 0.3884

2023-02-26 20:44:38.852936: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


31/31 [==============================] - 16s 380ms/step - loss: 1.7747 - categorical_accuracy: 0.2170 - precision_1: 0.1878 - recall_1: 0.3853 - top_k_categorical_accuracy: 0.3884 - val_loss: 1.7530 - val_categorical_accuracy: 0.2282 - val_precision_1: 0.2211 - val_recall_1: 0.5560 - val_top_k_categorical_accuracy: 0.4689
Epoch 2/80
31/31 [==============================] - 8s 253ms/step - loss: 1.7104 - categorical_accuracy: 0.3229 - precision_1: 0.2462 - recall_1: 0.4507 - top_k_categorical_accuracy: 0.5275 - val_loss: 1.7580 - val_categorical_accuracy: 0.2490 - val_precision_1: 0.1947 - val_recall_1: 0.5436 - val_top_k_categorical_accuracy: 0.4232
Epoch 3/80
31/31 [==============================] - 8s 259ms/step - loss: 1.6586 - categorical_accuracy: 0.4071 - precision_1: 0.2868 - recall_1: 0.5286 - top_k_categorical_accuracy: 0.5961 - val_loss: 1.7743 - val_categorical_accuracy: 0.2116 - val_precision_1: 0.1840 - val_recall_1: 0.4108 - val_top_k_categorical_accuracy: 0.3568
Epoch 4/

KeyboardInterrupt: 